# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 1:52PM,249621,10,PRF-40532,Bio-PRF,Released
1,Oct 28 2022 1:52PM,249621,10,PRF-40535,Bio-PRF,Released
2,Oct 28 2022 1:52PM,249621,10,PRF-40538,Bio-PRF,Released
3,Oct 28 2022 1:52PM,249621,10,PRF-40541,Bio-PRF,Released
4,Oct 28 2022 1:51PM,249622,10,SNAP1028221,"Snap Medical Industries, LLC",Released
5,Oct 28 2022 1:51PM,249622,10,SNAP32688435,"Snap Medical Industries, LLC",Released
6,Oct 28 2022 1:51PM,249622,10,SNAPA1933216,"Snap Medical Industries, LLC",Released
7,Oct 28 2022 1:51PM,249622,10,SNAPF2540546,"Snap Medical Industries, LLC",Released
8,Oct 28 2022 1:42PM,249620,10,8704500,"Citieffe, Inc.",Executing
9,Oct 28 2022 1:34PM,249619,10,0086126324,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,249604,Released,1
11,249619,Released,8
12,249620,Executing,1
13,249621,Released,4
14,249622,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249604,NaN,NaN,1.0
249619,NaN,NaN,8.0
249620,NaN,1.0,NaN
249621,NaN,NaN,4.0
249622,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249572,10.0,11.0,7.0
249587,4.0,0.0,1.0
249592,0.0,0.0,1.0
249593,0.0,2.0,1.0
249594,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249572,10,11,7
249587,4,0,1
249592,0,0,1
249593,0,2,1
249594,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249572,10,11,7
1,249587,4,0,1
2,249592,0,0,1
3,249593,0,2,1
4,249594,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249572,10,11,7
1,249587,4,,1
2,249592,,,1
3,249593,,2,1
4,249594,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 1:52PM,249621,10,Bio-PRF
4,Oct 28 2022 1:51PM,249622,10,"Snap Medical Industries, LLC"
8,Oct 28 2022 1:42PM,249620,10,"Citieffe, Inc."
9,Oct 28 2022 1:34PM,249619,10,ISDIN Corporation
17,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc."
18,Oct 28 2022 10:50AM,249603,19,"Emersa Waterbox, LLC"
19,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC"
20,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc."
23,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC"
24,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 28 2022 1:52PM,249621,10,Bio-PRF,,,4
1,Oct 28 2022 1:51PM,249622,10,"Snap Medical Industries, LLC",,,4
2,Oct 28 2022 1:42PM,249620,10,"Citieffe, Inc.",,1,
3,Oct 28 2022 1:34PM,249619,10,ISDIN Corporation,,,8
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",,,1
5,Oct 28 2022 10:50AM,249603,19,"Emersa Waterbox, LLC",,,1
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",,,1
7,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc.",,2,1
8,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC",,,1
9,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,4,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 1:52PM,249621,10,Bio-PRF,4,,
1,Oct 28 2022 1:51PM,249622,10,"Snap Medical Industries, LLC",4,,
2,Oct 28 2022 1:42PM,249620,10,"Citieffe, Inc.",,1,
3,Oct 28 2022 1:34PM,249619,10,ISDIN Corporation,8,,
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1,,
5,Oct 28 2022 10:50AM,249603,19,"Emersa Waterbox, LLC",1,,
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1,,
7,Oct 28 2022 10:14AM,249593,15,"Alliance Pharma, Inc.",1,2,
8,Oct 28 2022 10:14AM,249592,15,"Brookfield Pharmaceuticals, LLC",1,,
9,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,1,,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 1:52PM,249621,10,Bio-PRF,4,,
1,Oct 28 2022 1:51PM,249622,10,"Snap Medical Industries, LLC",4,,
2,Oct 28 2022 1:42PM,249620,10,"Citieffe, Inc.",,1,
3,Oct 28 2022 1:34PM,249619,10,ISDIN Corporation,8,,
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 1:52PM,249621,10,Bio-PRF,4.0,NaN,NaN
1,Oct 28 2022 1:51PM,249622,10,"Snap Medical Industries, LLC",4.0,NaN,NaN
2,Oct 28 2022 1:42PM,249620,10,"Citieffe, Inc.",NaN,1.0,NaN
3,Oct 28 2022 1:34PM,249619,10,ISDIN Corporation,8.0,NaN,NaN
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 28 2022 1:52PM,249621,10,Bio-PRF,4.0,0.0,0.0
1,Oct 28 2022 1:51PM,249622,10,"Snap Medical Industries, LLC",4.0,0.0,0.0
2,Oct 28 2022 1:42PM,249620,10,"Citieffe, Inc.",0.0,1.0,0.0
3,Oct 28 2022 1:34PM,249619,10,ISDIN Corporation,8.0,0.0,0.0
4,Oct 28 2022 10:58AM,249604,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,998482,16.0,1.0,0.0
12,249594,1.0,0.0,0.0
15,748757,9.0,13.0,10.0
16,249587,1.0,0.0,4.0
19,499207,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,998482,16.0,1.0,0.0
1,12,249594,1.0,0.0,0.0
2,15,748757,9.0,13.0,10.0
3,16,249587,1.0,0.0,4.0
4,19,499207,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,16.0,1.0,0.0
1,12,1.0,0.0,0.0
2,15,9.0,13.0,10.0
3,16,1.0,0.0,4.0
4,19,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,16.0
1,12,Released,1.0
2,15,Released,9.0
3,16,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19
Status,,,,,
Completed,0.0,0.0,10.0,4.0,0.0
Executing,1.0,0.0,13.0,0.0,0.0
Released,16.0,1.0,9.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19
0,Completed,0.0,0.0,10.0,4.0,0.0
1,Executing,1.0,0.0,13.0,0.0,0.0
2,Released,16.0,1.0,9.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19
0,Completed,0.0,0.0,10.0,4.0,0.0
1,Executing,1.0,0.0,13.0,0.0,0.0
2,Released,16.0,1.0,9.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()